In [1]:
# 1. import any required library to load dataset, open files (os), print confusion matrix and accuracy score
import pickle
import pandas as pd
import numpy as np
import os
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


In [2]:
# 2. Create test set if you like to do the split programmatically or if you have not already split the data at this point

# Load dataset
df = pd.read_csv('disadvantaged_communities.csv')

#encode categorical variables...
from category_encoders import OrdinalEncoder
# Define categorical columns to be encoded
categorical_cols = df.columns.drop(['CES 4.0 Percentile Range'])  # append all cols in list except target variable column
# Create an instance of OrdinalEncoder
encoder = OrdinalEncoder(cols=categorical_cols)
# Fit and transform the categorical variables in the DataFrame
df_encoded = encoder.fit_transform(df)

#split data (90% 10% split for evaluation purpose.  only going to use the 10%)

# Separate predictor variables from the target variable (attributes (X) and target variable (y) as we did in the class)
X = df_encoded.drop(columns=['CES 4.0 Percentile Range'])
# Select the target column
y = df_encoded['CES 4.0 Percentile Range']  # All rows, column 9


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)

C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtyp

In [3]:
# 3. Load your saved model for dissadvantaged communities classification 
#that you saved in dissadvantaged_communities_classification.py via Pikcle

# Load the models from the file
with open('SvmClassifier.sav', 'rb') as file:
    svm = pickle.load(file)

with open('RfClassifier.sav', 'rb') as file:
    rf = pickle.load(file)


In [4]:
# 4. Make predictions on test_set created from step 2
rfPredictions = rf.predict(X_test)


#NOTE: FOR SVM, NEED TO STANDARDIZE THE FEAUTURES FIRST
from sklearn.preprocessing import StandardScaler
# Create an instance of StandardScaler
scaler = StandardScaler()
# Fit the scaler to the training set and transform it
X_test_scaled = scaler.fit_transform(X_test)

svmPredictions = svm.predict(X_test_scaled)

In [5]:
# 5. use predictions and test_set (X_test) classifications to print the following:
#    1. confution matrix, 2. accuracy score, 3. precision, 4. recall, 5. specificity
#    You can easily find the formulae for Precision, Recall, and Specificity online.

# Compute the confusion matrix
cm_rf = confusion_matrix(y_test, rfPredictions)
cm_svm = confusion_matrix(y_test, svmPredictions)

# Compute the accuracy score
accuracy_rf = accuracy_score(y_test, rfPredictions)
accuracy_svm = accuracy_score(y_test, svmPredictions)

# Compute the precision
precision_rf = precision_score(y_test, rfPredictions, average='weighted')
precision_svm = precision_score(y_test, svmPredictions, average='weighted')

# Compute the recall
recall_rf = recall_score(y_test, rfPredictions, average='weighted')
recall_svm = recall_score(y_test, svmPredictions, average='weighted')

# Compute the specificity for each class and take the average
specificity_rf = np.mean([cm_rf[i,i]/np.sum(cm_rf[:,i]) for i in range(cm_rf.shape[0])])
specificity_svm = np.mean([cm_svm[i,i]/np.sum(cm_svm[:,i]) for i in range(cm_svm.shape[0])])


C:\Users\godha\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
# Print the metrics
print(f"Random Forest Metrics:\nConfusion Matrix:\n{cm_rf}\nAccuracy: {accuracy_rf}\nPrecision: {precision_rf}\nRecall: {recall_rf}\nSpecificity: {specificity_rf}\n")

Random Forest Metrics:
Confusion Matrix:
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0]
 [ 1  3  4  1  0  0  0  0  0  0]
 [ 0  0  0  6  9  0  0  0  0  0]
 [ 0  0  0  0  6 34  0  0  0  0]
 [ 0  0  0  0  0  5 35  0  0  0]
 [ 0  0  0  0  0  0  3 37  0  0]
 [ 0  0  0  0  0  0  0  3 37  0]
 [ 0  0  0  0  0  0  0  0  0 40]]
Accuracy: 0.8427947598253275
Precision: 0.8689939494352273
Recall: 0.8427947598253275
Specificity: 0.5460704646230962



In [7]:
# Print the metrics
print(f"SVM Metrics:\nConfusion Matrix:\n{cm_svm}\nAccuracy: {accuracy_svm}\nPrecision: {precision_svm}\nRecall: {recall_svm}\nSpecificity: {specificity_svm}\n")

SVM Metrics:
Confusion Matrix:
[[ 2  0  0  0  0  0  0  0  0]
 [ 1  1  1  0  0  0  0  0  0]
 [ 0  1  8  0  0  0  0  0  0]
 [ 0  0  1 14  0  0  0  0  0]
 [ 0  0  0  0 39  1  0  0  0]
 [ 0  0  0  0  0 39  1  0  0]
 [ 0  0  0  0  0  1 39  0  0]
 [ 0  0  0  0  0  0  0 38  2]
 [ 0  0  0  0  0  0  0  0 40]]
Accuracy: 0.9606986899563319
Precision: 0.961473152473259
Recall: 0.9606986899563319
Specificity: 0.8716963479158601

